# Prerequsites

## Install dependencies

In [ ]:
!pip install pandas ipython-sql


In [ ]:
!pip install beautifulsoup4 requests

## Setup Logging

In [ ]:
import logging

logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

## Add imports

In [ ]:
import pandas as pd

In [ ]:
from bs4 import BeautifulSoup
import requests

# Create Datasets for all pharmacies in London separated by borough

## Download pharmacy list from https://opendata.nhsbsa.net/dataset/consolidated-pharmaceutical-list

In [ ]:
# Consolidated Pharmaceutical List: 2023-24 Q4
# https://opendata.nhsbsa.net/dataset/240d142d-df82-4e97-b051-12371519e4e1/resource/d36c355a-631d-4ddb-bc34-5a0315565e12/download/consol_pharmacy_list_202324q4.csv

CONSOLIDATED_PHARMACEUTICAL_LIST_CSV = "https://opendata.nhsbsa.net/dataset/240d142d-df82-4e97-b051-12371519e4e1/resource/d36c355a-631d-4ddb-bc34-5a0315565e12/download/consol_pharmacy_list_202324q4.csv"
pharmacy_df = pd.read_csv(CONSOLIDATED_PHARMACEUTICAL_LIST_CSV)

## Generate London Pharmacies Dataframe

In [ ]:
lon_pharmacy_df = pharmacy_df[["PHARMACY_ODS_CODE_(F-CODE)", "HEALTH_AND_WELLBEING_BOARD", "PHARMACY_TRADING_NAME", "ADDRESS_FIELD1", "ADDRESS_FIELD2", "ADDRESS_FIELD3", "ADDRESS_FIELD4", "POST_CODE"]]

In [ ]:
lon_health_wellbeing_boards = ["CITY OF LONDON", "KENSINGTON AND CHELSEA", "BRENT", "HAMMERSMITH AND FULHAM", "HOUNSLOW", "ISLINGTON", "SUTTON", "WALTHAM FOREST", "KINGSTON", "BARNET", "SOUTHWARK", "CAMDEN", "HILLINGDON", "WESTMINSTER", "CROYDON", "HARINGEY", "GREENWICH", "LEWISHAM", "BARKING AND DAGENHAM", "MERTON", "LAMBETH", "EALING", "BEXLEY", "HARROW", "REDBRIDGE", "HAVERING", "HACKNEY", "TOWER HAMLETS"]
boroughs = []

In [ ]:
lon_pharmacy_df = lon_pharmacy_df.loc[pharmacy_df.HEALTH_AND_WELLBEING_BOARD.isin(lon_health_wellbeing_boards)]

## Create Dataframe for each London borough from London Pharmacies Dataframe

In [ ]:
def create_borough_df(borough_name):
    return lon_pharmacy_df.loc[lon_pharmacy_df.HEALTH_AND_WELLBEING_BOARD.isin([borough_name])]

In [ ]:
for health_board in lon_health_wellbeing_boards:
    boroughs.append(
        {
            "health_board": health_board,
            "df": create_borough_df(health_board)
        }
    )

# Create CSV with phone numbers

In [ ]:
def get_phone_number(pharmacy_ods_code, pharmacy_trading_name):
    
    pharmacy_trading_name = pharmacy_trading_name.lower()
    pharmacy_trading_name = pharmacy_trading_name.replace(" ", "-")
    
    query = f"https://www.nhs.uk/services/pharmacy/{pharmacy_trading_name}/{pharmacy_ods_code}"
    logging.info(f"Calling URI {query}...")
    response = requests.get(query)
    resp_status_code = response.status_code
    
    if response.status_code != 200:
        logging.error(f"Query failed with response code {resp_status_code}. ODS Code: {pharmacy_ods_code}. Writing empty phone number...")
        return ""
    
    logging.info(f"Query successful with response code {resp_status_code}")
    soup = BeautifulSoup(response.text, 'html.parser')
    try:
        phone_number = soup.find("p", {"id": "contact_info_panel_phone_text"}).getText()
    except:
        logging.error(f"Soup could not find a phone number for pharmacy {pharmacy_trading_name}, {pharmacy_ods_code} at URI {query}...")
        return ""
    return phone_number

def phone_number_with_country_code_and_no_spaces(phone_number):
    phone_number = phone_number.replace(" ", "")
    return phone_number.replace("0", "+44", 1)

In [ ]:
def add_phone_numbers_for_df(df):
    phone_numbers = []

    row_count = df.shape[0]
    i = 0
    
    while i < row_count:
        ods_code, pharmacy_name = df[["PHARMACY_ODS_CODE_(F-CODE)", "PHARMACY_TRADING_NAME"]].iloc[i]
        phone_number = get_phone_number(ods_code, pharmacy_name)
        phone_number = phone_number_with_country_code_and_no_spaces(phone_number)
        phone_numbers.append(phone_number)
        i += 1
    
    df.loc[:, "PHONE_NUMBER"] = phone_numbers

In [ ]:
def generate_csv_name(health_board):
    return health_board.lower().title().replace(" ", "_") + '_Pharmacies.csv'

def create_csv(df, name):
    path = './' + name
    df.to_csv(path, index=False)
    logging.info(f"CSV successfully created at path: {path}...")

In [ ]:
pd.options.mode.chained_assignment = None

for borough_dict in boroughs:
    df = borough_dict.get('df')
    add_phone_numbers_for_df(df)
    df = df.drop("HEALTH_AND_WELLBEING_BOARD", axis=1)
    create_csv(df, generate_csv_name(borough_dict.get('health_board')))

In [ ]:
logging.info("Job complete!")